In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%config Completer.use_jedi = False

In [3]:
!ls

Untitled.ipynb __pycache__    train.txt


In [5]:
df = pd.read_pickle("../resources/cleaned_data_fasttext.pickle")

In [7]:
type(df["fasttext_embedding"][0])

numpy.ndarray

In [8]:
df.head()

,url,cleaned_important_text,embedding,publish_datetime,polarity,subjectivity,fasttext_embedding
0,https://www.rt.com/usa/520711-whitmer-aide-spr...,michigan governor office dismiss partisan repo...,"[-0.08033423870801926, 0.10895209014415741, 0....",2021-04-11T11:20:00.000Z,Positive,Neutral,"[-0.26263347, 0.2678325, -0.26754516, 0.017797..."
1,https://thenextweb.com/news/should-countries-e...,country respond cyberattack physical force ind...,"[-0.3013652265071869, 0.23083247244358063, 0.1...",2021-04-11T11:51:13.000Z,Neutral,Very objective,"[-0.25623396, 0.27985862, -0.2651556, 0.014549..."
2,https://www.cnet.com/news/nasa-stares-into-dar...,nasa discover dark freaky pit mars stare shado...,"[0.09577909111976624, -0.20507732033729553, 0....",2021-04-11T01:48:43.000Z,Neutral,Neutral,"[-0.23300028, 0.23795356, -0.23213726, 0.01760..."
3,https://abc7ny.com/10506004/?ex_cid=TA_WABC_TW...,second weekly pop performance hold broadway th...,"[0.22969390451908112, 0.10592672973871231, 0.1...",2021-04-11T00:05:06.000Z,Neutral,Very objective,"[-0.25241378, 0.26854432, -0.25793087, 0.01818..."
4,https://thenextweb.com/news/perfecting-self-dr...,go perfect self drive car independent news com...,"[-0.1521698385477066, 0.22504617273807526, -0....",2021-04-11T09:46:12.000Z,Very positive,Very subjective,"[-0.2680339, 0.2807403, -0.27258533, 0.0205178..."


In [9]:
import fasttext

In [10]:
model = fasttext.load_model("../resources/model.bin")

In [18]:
model.get_labels(include_freq=True)

1950

In [40]:
model.get_nearest_neighbors("asian")

[(0.9998489022254944, 'urban'),
 (0.9998421669006348, 'iran'),
 (0.9998250603675842, 'indian'),
 (0.999823272228241, 'japan'),
 (0.999803900718689, 'hard'),
 (0.999801754951477, 'democratic'),
 (0.9998009204864502, 'familiar'),
 (0.9997973442077637, 'harvard'),
 (0.9997968673706055, 'democracy'),
 (0.999796450138092, 'factor')]

In [54]:
new_model = fasttext.train_unsupervised("train.txt", "cbow", lr=0.05, epoch=500)

In [55]:
new_model.get_nearest_neighbors("philip")

[(0.5153183341026306, 'trip'),
 (0.3442321717739105, 'phase'),
 (0.34157735109329224, 'chile'),
 (0.3285171091556549, 'likely'),
 (0.3213077485561371, 'facility'),
 (0.318682461977005, 'ship'),
 (0.2958216667175293, 'summer'),
 (0.2920197546482086, 'smoke'),
 (0.27656933665275574, 'volunteer'),
 (0.2675592005252838, 'rover')]

In [1]:
import spacy

In [2]:
NLP = spacy.load("en_core_web_lg")

In [9]:
text = "asian united states trump biden crime rasicm msnbc"

In [10]:
target_doc = NLP(text)

In [11]:
target_doc = NLP(text)
target_entities = [(X.text, X.label_) for X in target_doc.ents]

print("doc", target_doc)
print("ents", target_entities)

doc asian united states Trump Biden crime rasicm msnbc
ents [('asian', 'NORP'), ('Biden', 'PERSON'), ('msnbc', 'ORG')]


In [5]:
import re

In [6]:
test_input = df.iloc[0].cleaned_important_text

In [7]:
test_input

'triple decker 21st century airtight solar power stand level house somerville lena sheehan look construction new high school transportation hub block away \n\n long say new t brilliant right house \n\n immigrate ireland early 90 sheehan husband buy property school street 1998 advice friend \n\n crazy idea somerville place say'

In [8]:
test_input = re.sub("\n", "", test_input)
test_input = re.sub("\t", "", test_input)

In [9]:
" ".join(test_input.split())

'triple decker 21st century airtight solar power stand level house somerville lena sheehan look construction new high school transportation hub block away long say new t brilliant right house immigrate ireland early 90 sheehan husband buy property school street 1998 advice friend crazy idea somerville place say'

In [10]:
def remove_spaces(text):
    text = re.sub("\n", "", text)
    text = re.sub("\t", "", text)
    text = " ".join(text.split())
    return text

In [11]:
df["cleaned_important_text"] = df.cleaned_important_text.apply(remove_spaces)

In [12]:
df.to_csv("resources/cleaned_data_bert.csv")

In [13]:
cached = pd.DataFrame(df["cleaned_important_text"])

In [15]:
cached.reset_index(drop=True, inplace=True)

In [17]:
np.savetxt('./resources/train.txt', cached.values, fmt="%s")

In [18]:
import fasttext

In [19]:
model = fasttext.train_unsupervised('./resources/train.txt')

In [20]:
model.save_model("./resources/model.bin")

In [33]:
def get_mean_embedding(text):
    emb = np.mean([model.get_word_vector(word) for word in text.split()], axis=0)
    return emb

def calculate_embeddings():
    if model is None:
        self.__logger.error("Need to train model first")
        return

    df["fasttext_embedding"] = df.cleaned_important_text.apply(lambda text: get_mean_embedding(str(text)))

    df.to_pickle('./resources/cleaned_data_fasttext.pickle')
    return df

In [34]:
test = calculate_embeddings()

In [36]:
test_input

'triple decker 21st century airtight solar power stand level house somerville lena sheehan look construction new high school transportation hub block away  long say new t brilliant right house  immigrate ireland early 90 sheehan husband buy property school street 1998 advice friend  crazy idea somerville place say'

In [39]:
model.get_nearest_neighbors("suez")

[(0.8565452694892883, 'shipwreck'),
 (0.8223105072975159, 'nike'),
 (0.8194195032119751, 'suzhou'),
 (0.8192678689956665, 'subway'),
 (0.8159365653991699, 'fireball'),
 (0.8148849010467529, 'nickel'),
 (0.813416063785553, 'ddt'),
 (0.8080143332481384, 'supernova'),
 (0.8031246662139893, 'nile'),
 (0.8000850677490234, 'sunny')]

In [32]:
test

,url,cleaned_important_text,embedding,publish_datetime,polarity,subjectivity,bert_embedding,fasttext_embedding
0,https://www.wbur.org/earthwhile/2021/03/12/tri...,triple decker 21st century airtight solar powe...,"[-0.012675878591835499, 0.232682004570961, -0....",2021-03-21T22:04:29.000Z,Neutral,Very objective,"tensor([[ 0.7663, 1.2018, -0.0192, ..., -0.3...","[0.20896038, -0.03249417, -0.046633776, 0.1013..."
1,https://www.forbes.com/sites/edwardsegal/2021/...,covid continue employee remain concerned retur...,"[-0.08585719019174576, 0.21065688133239746, 0....",2021-03-21T11:00:07.000Z,Positive,Neutral,"tensor([[ 0.9921, -0.1721, -0.7232, ..., -0.2...","[0.21479964, 0.06367354, -0.044154134, 0.14962..."
2,https://slate.com/technology/2021/03/japanese-...,non english edition wikipedia misinformation p...,"[-0.15740007162094116, 0.06153416261076927, 0....",2021-03-21T17:19:38.000Z,Neutral,Very objective,"tensor([[ 0.5851, 0.7159, 0.0848, ..., 1.2...","[0.13818005, 0.073198095, -0.038398672, 0.0972..."
3,https://abc7news.com/10434159/?ex_cid=TA_KGO_T...,woman likely experience effect covid-19 vaccin...,"[-0.14862942695617676, 0.20998084545135498, -0...",2021-03-21T07:45:02.000Z,Neutral,Subjective,"tensor([[ 0.5773, -0.4211, -1.1366, ..., -0.0...","[0.14626163, 0.13944134, -0.13908085, 0.058073..."
4,https://www.cbsnews.com/news/iceland-volcano-f...,"volcano dormant 6,000 year erupt iceland icela...","[0.1105966717004776, -0.06708431988954544, 0.0...",2021-03-21T13:34:02.000Z,Neutral,Very objective,"tensor([[ 0.2277, -0.7162, -0.9377, ..., -1.4...","[0.18304849, 0.058092527, 0.047184613, 0.08407..."
...,...,...,...,...,...,...,...,...
7635,https://www.nature.com/articles/d41586-021-008...,world learn covid dive pandemic treaty week 27...,"[-0.1349020004272461, 0.047990813851356506, 0....",2021-04-06T15:03:42.000Z,Neutral,Very objective,"tensor([[ 0.5842, 0.0834, -0.9654, ..., 1.2...","[0.2034998, -0.06291013, 0.0001865788, 0.15545..."
7636,https://www.bloomberg.com/news/articles/2021-0...,bitcoin mining operation china threaten climat...,"[-0.1503271609544754, 0.21208292245864868, 0.1...",2021-04-06T16:34:05.000Z,Neutral,Very objective,"tensor([[ 1.0149, 1.5701, 0.4140, ..., 0.7...","[0.1323934, 0.060118645, 0.031312052, 0.213567..."
7637,https://www.newstatesman.com/business/work/202...,mass remote working set uk employment skill cr...,"[-0.1262105256319046, 0.10679781436920166, 0.0...",2021-04-06T17:45:05.000Z,Neutral,Objective,"tensor([[ 1.0571, 0.1128, -0.2033, ..., -0.3...","[0.17134072, 0.0293745, -0.09532484, 0.1025412..."
7638,https://abc7ny.com/health/covid-updates-rise-i...,coronavirus vaccine update cdc say rise case v...,"[-0.15127120912075043, 0.2799147367477417, 0.0...",2021-04-06T21:15:08.000Z,Neutral,Objective,"tensor([[ 1.0091, 0.3105, -0.4188, ..., 0.1...","[0.253463, -0.058295492, -0.2395827, 0.0012964..."
